<a href="https://colab.research.google.com/github/gbessardon/LAI_treatement/blob/main/Create_ECO-SG_chunk/Create_ecosg_tif_chunk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# if running in google colab

In [1]:
colab=1
if colab==1:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Declare libraries

In [2]:
import gdal
from osgeo import gdalconst

# Declare input/output filenames

In [3]:
ECOSGinput='/content/drive/MyDrive/ECOCLIMAP_SG/ecosg_final_map.dir'
MODISinput='/content/drive/MyDrive/MODIS_bolli/modis_lai_mym_v2_2012-2021_07.28.tif'# MODIS data

ECOSGoutput='/content/drive/MyDrive/ECOCLIMAP_SG/ecosgcut.tif'
MODISoutput='/content/drive/MyDrive/MODIS_bolli/modis_repojected.tif'

# Declare function

## Function to extract modis file extent

In [4]:
def extractextent(fnmodis):
  jsoninfo=gdal.Info(fnmodis,format='json')
  EXTENT=jsoninfo.get('wgs84Extent')
  xmax,ymax=EXTENT.get('coordinates')[0][0]
  xmin,ymin=EXTENT.get('coordinates')[0][1]
  return(xmax,ymax,xmin,ymin)

## Function to cut ECOSG betwen latmin,latmax,lonmin,lonmax

In [22]:
def cutSG(fnSG,outfn,latmin=79,latmax=89.99,
          lonmin=-180,lonmax=90,proj='EPSG:4326'):
  src_fn=fnSG
  ds=gdal.Open(src_fn)
  gdal.Warp(outfn,ds,
            outputBounds =(lonmin,latmin,lonmax,latmax), 
            dstSRS=proj)

## Function to reproject MODISfile following the ECOSG cutted file

In [6]:
 def reproject_modis(MODISfile,fnecosg,dst_filename):  
    
    src=gdal.Open(MODISfile)
    src_proj = src.GetProjection()
    src_geotrans = src.GetGeoTransform()

    #Open the ecosg file to get the projection details
    match_filename = fnecosg
    match_ds = gdal.Open(match_filename)
    match_proj = match_ds.GetProjection()
    match_geotrans = match_ds.GetGeoTransform()
    wide = match_ds.RasterXSize
    high = match_ds.RasterYSize

    #Create the MODIS tiff file with the right projection
    dst = gdal.GetDriverByName('GTiff').Create(dst_filename, wide, high, 1, gdal.GDT_Byte)
    dst.SetGeoTransform( match_geotrans )
    dst.SetProjection( match_proj)
    gdal.ReprojectImage(src, dst, src_proj, match_proj, gdalconst.GRA_Bilinear)

    del dst
    return

# Main

## Extract modis file extent

In [7]:
(xmax,ymax,xmin,ymin)=extractextent(MODISinput)

## Cut ECO-SG between modis extent

In [23]:
cutSG(ECOSGinput,ECOSGoutput,latmin=ymin,latmax=ymax,
          lonmin=xmin,lonmax=xmax)

# Reproject modis

In [24]:
reproject_modis(MODISinput,ECOSGoutput,MODISoutput)

# Check output

In [25]:
gdal.Info(MODISoutput,format='json')

{'bands': [{'band': 1,
   'block': [84451, 1],
   'colorInterpretation': 'Gray',
   'metadata': {},
   'type': 'Byte'}],
 'coordinateSystem': {'wkt': 'GEOGCS["WGS 84",\n    DATUM["WGS_1984",\n        SPHEROID["WGS 84",6378137,298.257223563,\n            AUTHORITY["EPSG","7030"]],\n        AUTHORITY["EPSG","6326"]],\n    PRIMEM["Greenwich",0],\n    UNIT["degree",0.0174532925199433],\n    AUTHORITY["EPSG","4326"]]'},
 'cornerCoordinates': {'center': [55.0647343, 70.0],
  'lowerLeft': [-62.2289531, 50.0],
  'lowerRight': [172.3584217, 50.0],
  'upperLeft': [-62.2289531, 90.0],
  'upperRight': [172.3584217, 90.0]},
 'description': '/content/drive/MyDrive/MODIS_bolli/modis_repojected.tif',
 'driverLongName': 'GeoTIFF',
 'driverShortName': 'GTiff',
 'files': ['/content/drive/MyDrive/MODIS_bolli/modis_repojected.tif'],
 'geoTransform': [-62.2289531,
  0.002777792741353,
  0.0,
  90.0,
  0.0,
  -0.0027777777777778],
 'metadata': {'': {'AREA_OR_POINT': 'Area'},
  'IMAGE_STRUCTURE': {'INTERLEAVE

In [26]:
gdal.Info(ECOSGoutput,format='json')

{'bands': [{'band': 1,
   'block': [84451, 1],
   'colorInterpretation': 'Gray',
   'metadata': {},
   'noDataValue': 0.0,
   'type': 'Byte'}],
 'coordinateSystem': {'wkt': 'GEOGCS["WGS 84",\n    DATUM["WGS_1984",\n        SPHEROID["WGS 84",6378137,298.257223563,\n            AUTHORITY["EPSG","7030"]],\n        AUTHORITY["EPSG","6326"]],\n    PRIMEM["Greenwich",0],\n    UNIT["degree",0.0174532925199433],\n    AUTHORITY["EPSG","4326"]]'},
 'cornerCoordinates': {'center': [55.0647343, 70.0],
  'lowerLeft': [-62.2289531, 50.0],
  'lowerRight': [172.3584217, 50.0],
  'upperLeft': [-62.2289531, 90.0],
  'upperRight': [172.3584217, 90.0]},
 'description': '/content/drive/MyDrive/ECOCLIMAP_SG/ecosgcut.tif',
 'driverLongName': 'GeoTIFF',
 'driverShortName': 'GTiff',
 'files': ['/content/drive/MyDrive/ECOCLIMAP_SG/ecosgcut.tif'],
 'geoTransform': [-62.2289531,
  0.002777792741353,
  0.0,
  90.0,
  0.0,
  -0.0027777777777778],
 'metadata': {'': {'AREA_OR_POINT': 'Area'},
  'IMAGE_STRUCTURE': {'I